# Specific User Recommendation

## I. Setup


In [1]:
import sys
sys.path.append('..')


In [2]:
from lib.types.dataset_type import DatasetType
from lib.types.source_type import SourceType
from lib.utils.utils import print_dict
from lib.services.user_interest_service import UserInterestService
from models.reRank_model import reRank_model

import pandas as pd


## II. Get user interest genres


In [7]:
service = UserInterestService(user_id=123963, source_prefix="..")
interest_dict = service.exec()
interest_dict


/Users/theachoem/Coding/Pythons/movie_recommendation/notebooks/../lib/services/user_interest_service.py:33: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(
/Users/theachoem/Coding/Pythons/movie_recommendation/notebooks/../lib/services/user_interest_service.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_per_user_df['movie_id'] = rating_per_user_df['movie_id'].astype(


{'TV Movie': 4.0,
 'Adventure': 3.875,
 'Foreign': 3.6666666666666665,
 'Western': 3.5,
 'History': 3.5,
 'Romance': 3.4642857142857144,
 'Horror': 3.4444444444444446,
 'Fantasy': 3.375,
 'Science Fiction': 3.35,
 'Drama': 3.3205128205128207,
 'Action': 3.2857142857142856,
 'Comedy': 3.272727272727273,
 'Crime': 3.2666666666666666,
 'Thriller': 3.261904761904762,
 'Mystery': 3.142857142857143,
 'Documentary': 2.5,
 'Family': 2.0,
 'Music': 2.0,
 'Animation': 0.5}

## III. Construct fake movie with favorite genres

### a. Construct genres in json format


In [59]:
import json as json

filtered_ratings = [
    {"name": genre, "id": rating} for genre,
    rating in interest_dict.items() if rating >= 3
]

genres = json.dumps(filtered_ratings)
genres


'[{"name": "TV Movie", "id": 4.0}, {"name": "Adventure", "id": 3.875}, {"name": "Foreign", "id": 3.6666666666666665}, {"name": "Western", "id": 3.5}, {"name": "History", "id": 3.5}, {"name": "Romance", "id": 3.4642857142857144}, {"name": "Horror", "id": 3.4444444444444446}, {"name": "Fantasy", "id": 3.375}, {"name": "Science Fiction", "id": 3.35}, {"name": "Drama", "id": 3.3205128205128207}, {"name": "Action", "id": 3.2857142857142856}, {"name": "Comedy", "id": 3.272727272727273}, {"name": "Crime", "id": 3.2666666666666666}, {"name": "Thriller", "id": 3.261904761904762}, {"name": "Mystery", "id": 3.142857142857143}]'

### b. Construct metadata


In [60]:
df = pd.read_csv(DatasetType.movies_metadata.original_path())
df.head(1)

columns = ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
           'imdb_id', 'original_language', 'original_title', 'overview',
           'popularity', 'poster_path', 'production_companies',
           'production_countries', 'release_date', 'revenue', 'runtime',
           'spoken_languages', 'status', 'tagline', 'title', 'video',
           'vote_average', 'vote_count']

row = []
for c_name in columns:
    if(c_name == "id"):
        row.append(0)
    elif(c_name == "genres"):
        row.append(genres)
    else:
        row.append("Nan")

interested_genres_df = pd.DataFrame(columns=columns, data=[row])
metadata_df = pd.concat([interested_genres_df, df])
metadata_df


/var/folders/g8/wgj905m158gbcnzm6q5v9y2r0000gn/T/ipykernel_5210/742018980.py:1: DtypeWarning: Columns (0,2,5,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DatasetType.movies_metadata.original_path())


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,Nan,Nan,Nan,"[{""name"": ""TV Movie"", ""id"": 4.0}, {""name"": ""Ad...",Nan,0,Nan,Nan,Nan,Nan,...,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45459,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45460,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45461,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45462,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


## IV. Train

In [61]:
from models.contentbased_model import contentbased_model

model = contentbased_model()
trained_data = model.trainingDataBasedOnGenres(metadata_df)


TypeError: __init__() missing 2 required positional arguments: 'movie_title' and 'metadata'